In [1]:
import numpy as np
import pandas as pd

from utils import read_pickles, WRMSSEEvaluator

pd.set_option('display.max_columns', 100)

%matplotlib inline

In [2]:
df = pd.read_csv('../input/sales_train_evaluation.csv')
calendar = pd.read_csv('../input/calendar.csv')
prices = pd.read_csv('../input/sell_prices.csv')

In [11]:
sub = pd.read_csv("../output/submission_lgbm_28days.csv") # 0.54652
sample_sub = pd.read_csv("../input/sample_submission.csv")

In [12]:
sample_sub["order"] = range(sample_sub.shape[0])

df_train = df.iloc[:, :-28]
df_valid = df.iloc[:, -28:]

evaluator = WRMSSEEvaluator(df_train, df_valid, calendar, prices)

100%|██████████| 12/12 [00:10<00:00,  1.13it/s]


In [13]:
cols_d = [f'd_{1914+i}' for i in range(0,28)]
sub.columns = ['id']+cols_d
sub = sub[sub['id'].str.contains("validation")]
sub = sub.merge(sample_sub[["id", "order"]], on = "id").sort_values("order").drop(["id", "order"], axis = 1).reset_index(drop = True)
groups, scores = evaluator.score(sub[cols_d])

In [16]:
np.mean(scores)

0.5280175729935999

In [22]:
oof = pd.read_csv('../output/oof_lgbm_cv.csv')

In [24]:
oof = oof.pivot(index='id', columns='d', values='demand').reset_index()

In [28]:
cols_d = [c for c in oof.columns if 'd_' in c]
oof

d,id,d_1000,d_1001,d_1002,d_1003,d_1004,d_1005,d_1006,d_1007,d_1008,d_1009,d_1010,d_1011,d_1012,d_1013,d_1014,d_1015,d_1016,d_1017,d_1018,d_1019,d_1020,d_1021,d_1022,d_1023,d_1024,d_1025,d_1026,d_1027,d_1028,d_1029,d_1030,d_1031,d_1032,d_1033,d_1034,d_1035,d_1036,d_1037,d_1038,d_1039,d_1040,d_1041,d_1042,d_1043,d_1044,d_1045,d_1046,d_1047,d_1048,...,d_950,d_951,d_952,d_953,d_954,d_955,d_956,d_957,d_958,d_959,d_960,d_961,d_962,d_963,d_964,d_965,d_966,d_967,d_968,d_969,d_970,d_971,d_972,d_973,d_974,d_975,d_976,d_977,d_978,d_979,d_980,d_981,d_982,d_983,d_984,d_985,d_986,d_987,d_988,d_989,d_990,d_991,d_992,d_993,d_994,d_995,d_996,d_997,d_998,d_999
0,FOODS_1_001_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,FOODS_1_001_CA_2_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,FOODS_1_001_CA_3_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,FOODS_1_001_CA_4_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,FOODS_1_001_TX_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,FOODS_1_001_TX_2_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,FOODS_1_001_TX_3_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,FOODS_1_001_WI_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [34]:
cols_valid = []
for c in cols_d:
    if (oof[c]>0).sum()!=0:
        cols_valid.append(c)

In [1]:
df

NameError: name 'df' is not defined

In [37]:
df_valid = df[cols_valid]
evaluator = WRMSSEEvaluator(df_train, df_valid, calendar, prices)

100%|██████████| 12/12 [00:11<00:00,  1.03it/s]


In [41]:
oof = oof.merge(sample_sub[["id", "order"]], on = "id").sort_values("order").drop(["id", "order"], axis = 1).reset_index(drop = True)
groups, scores = evaluator.score(oof[cols_valid])

In [43]:
oof[cols_valid]

,d_1549,d_1550,d_1551,d_1552,d_1553,d_1554,d_1555,d_1556,d_1557,d_1558,d_1559,d_1560,d_1561,d_1562,d_1563,d_1564,d_1565,d_1566,d_1567,d_1568,d_1569,d_1570,d_1571,d_1572,d_1573,d_1574,d_1575,d_1576,d_1858,d_1859,d_1860,d_1861,d_1862,d_1863,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,1.084843,0.809666,0.720794,0.715521,0.709955,0.839063,1.102379,0.913389,0.694978,0.697615,0.793450,0.778550,0.863777,0.995405,0.908528,0.784247,0.706450,0.800878,0.710872,0.764865,0.949677,0.887269,0.672904,0.608922,0.687501,0.696637,0.741043,1.000569,0.743859,0.780540,0.731524,0.747396,0.808643,0.962456,0.838490,0.734936,0.708264,0.599707,0.579967,0.674752,1.149038,1.058677,0.789664,0.681673,0.600801,0.614752,0.670497,1.109155,0.921403,0.948623,0.765547,0.790543,0.746845,0.879412,1.447120,1.201346,1.033151,0.982537,0.899609,0.923377,1.095846,1.444884,1.298398,0.957814,0.833176,0.823996,0.839918,1.063334,0.899715,0.818819,0.650570,0.766875,0.809737,0.823421,1.123069,1.297887,1.419760,1.136445,0.968251,1.009223,0.953188,1.074830,1.420138,1.379439
1,0.297156,0.184161,0.115449,0.115449,0.115449,0.255337,0.320210,0.315049,0.195556,0.147142,0.161118,0.161411,0.177748,0.264324,0.256690,0.160588,0.128019,0.128311,0.124751,0.233386,0.306460,0.304007,0.191841,0.152559,0.149377,0.149377,0.169274,0.261012,0.242673,0.225147,0.216463,0.220592,0.271751,0.332712,0.325963,0.252183,0.207331,0.208204,0.205902,0.247697,0.325309,0.327326,0.181562,0.190640,0.180065,0.166252,0.240102,0.387348,0.366952,0.229910,0.207706,0.205991,0.224935,0.280015,0.360763,0.260060,0.182651,0.147063,0.148884,0.188708,0.254874,0.331844,0.401792,0.286417,0.229248,0.229248,0.259496,0.311217,0.465273,0.424233,0.301076,0.247884,0.243990,0.241902,0.302137,0.356572,0.350123,0.226013,0.192598,0.195770,0.235490,0.321326,0.443128,0.501602
2,0.402914,0.249693,0.226714,0.215222,0.215222,0.307906,0.449370,0.417130,0.288475,0.227566,0.241542,0.212837,0.249815,0.390227,0.329864,0.223396,0.180938,0.181230,0.177670,0.233935,0.293382,0.278665,0.166499,0.132062,0.128879,0.128879,0.234246,0.311556,0.166193,0.165548,0.158600,0.163634,0.197936,0.282750,0.323897,0.183660,0.178609,0.190167,0.233435,0.375171,0.542603,0.507759,0.359329,0.293100,0.324004,0.286462,0.447198,0.524716,0.570420,0.423072,0.334236,0.326022,0.327698,0.377403,0.461669,0.319249,0.325956,0.282579,0.284400,0.290958,0.369512,0.943727,0.779192,0.602588,0.718406,0.651901,0.651901,0.785495,0.924911,0.813987,0.588161,0.548835,0.523409,0.513643,0.601999,0.780911,0.636931,0.469062,0.424618,0.424791,0.424791,0.533660,0.749588,0.682964
3,2.655703,1.857308,1.843813,1.793048,1.799915,2.281265,2.691438,3.525474,1.841682,1.767414,1.769044,2.403209,2.325224,2.524452,3.360715,2.293201,2.251629,2.392868,2.247959,2.401709,3.701780,3.654946,2.216636,2.036949,2.012682,2.221389,2.821389,3.118298,1.900001,1.884472,1.961724,2.256868,2.283155,2.605270,3.414489,2.418419,2.190045,2.179222,2.227056,2.555877,3.129717,3.545867,2.231978,1.862766,1.750823,1.721914,1.923522,2.754971,3.116400,1.791891,1.631114,1.650009,1.685913,2.065192,2.371511,2.459781,1.660421,1.523317,1.545316,1.579750,1.644197,1.964895,2.552699,1.662551,1.429758,1.273211,1.245128,1.348520,1.654595,1.774717,1.333381,1.139326,1.176424,1.296432,1.670153,2.094241,2.093326,1.489909,1.292100,1.503059,1.754668,1.686573,2.519680,3.071819
4,1.499705,1.030370,0.946920,0.923398,0.872419,1.004888,1.643622,1.400778,0.964382,0.886567,0.877131,0.938990,0.970406,1.360314,1.435391,1.132713,0.906092,0.922896,0.973369,1.123401,1.597741,1.712270,1.065136,0.998461,1.024354,0.982663,1.024324,1.419336,0.717301,0.955571,0.902071,0.965384,1.015954,1.285854,1.153475,0.854677,0.928485,0.842650,0.819842,0.870592,1.417206,1.413281